In [3]:
!pip install -U google-genai langgraph

In [4]:
import os
# replace with your actual key from Google AI Studio
os.environ["GEMINI_API_KEY"] = "API_Key"


In [5]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from google import genai
import os, time

# Initialize Gemini client (reads API key from environment)
client = genai.Client()

# ---- Safe wrapper to handle 503 errors ----
def safe_generate(prompt):
    for attempt in range(3):  # try up to 3 times
        try:
            response = client.models.generate_content(
                model="gemini-2.5-flash",   # ✅ use Flash for stability
                contents=prompt
            )
            return response.text
        except Exception as e:
            print(f"Error: {e}, retrying in 5s...")
            time.sleep(5)
    return "❌ Sorry, the server is too busy. Please try again later."

# ---- Define state (memory) ----
class ChatState(TypedDict):
    user_input: str
    history: List[str]
    response: str

# ---- Clarify Node ----

def clarify_node(state: ChatState) -> dict:
    user_input = state["user_input"].strip()

    # --- RULE 1: Empty input
    if not user_input:
        return {"response": "⚠️ Please type something.", "history": state["history"]}

    # --- RULE 2: One-character input → unclear
    if len(user_input) == 1:
        clarification = f"Can you please clarify what you mean by: '{user_input}'?"
        return {"response": clarification, "history": state["history"] + [f"Bot: {clarification}"]}

    # --- RULE 3: If input has 3+ words and ends with a question mark → treat as clear
    if len(user_input.split()) >= 3 and user_input.endswith("?"):
        return {"response": None, "history": state["history"]}

    # --- RULE 4: "I mean ..." → clarification
    if user_input.lower().startswith("i mean"):
        return {"response": None, "history": state["history"]}

    # --- Otherwise → ask Gemini
    result_text = safe_generate(
        f"Classify the following input as 'clear' or 'unclear'. "
        f"Answer ONLY with one word: clear OR unclear.\n\nInput: {user_input}"
    ).strip().lower()

    print("DEBUG clarify result:", result_text)

    if result_text == "unclear":
        clarification = f"Can you please clarify what you mean by: '{user_input}'?"
        return {"response": clarification, "history": state["history"] + [f"Bot: {clarification}"]}
    else:
        return {"response": None, "history": state["history"]}


# ---- Answer Node ----
def answer_node(state: ChatState) -> dict:
    user_input = state["user_input"]
    history = "\n".join(state["history"])

    response = safe_generate(
        f"Conversation so far:\n{history}\nUser: {user_input}\nBot:"
    )

    return {"response": response, "history": state["history"] + [f"User: {user_input}", f"Bot: {response}"]}

# ---- Build LangGraph ----
graph = StateGraph(ChatState)
graph.add_node("clarify", clarify_node)
graph.add_node("answer", answer_node)

graph.add_conditional_edges(
    "clarify",
    lambda state: "answer" if state["response"] is None else END
)
graph.add_edge("answer", END)
graph.set_entry_point("clarify")

chatbot = graph.compile()

# ---- Chat function for Colab ----
def chat_with_bot(user_input, history=[]):
    state = {"user_input": user_input, "history": history, "response": None}
    result = chatbot.invoke(state)
    return result["response"], result["history"]


In [9]:
history = []

print(" Ask a question! (Type 'exit' to stop)\n")

while True:
    user_input = input("You: ").strip()

    if not user_input:
        continue  # skip empty input

    if user_input.lower() == "exit":
        print("Bot: 👋 Chat ended. Thanks for talking with me!")
        break

    resp, history = chat_with_bot(user_input, history)
    print("Bot:", resp)
    print("-" * 50)


 Ask a question! (Type 'exit' to stop)

You: What is AI?
Bot: AI stands for **Artificial Intelligence**.

At its core, AI is a field of computer science focused on creating machines that can perform tasks that typically require human intelligence. This involves teaching computers to "think" or "reason" in ways that mimic human cognitive abilities.

This includes things like:

*   **Learning:** Adapting and improving based on data without being explicitly programmed for every scenario.
*   **Problem-solving:** Finding solutions to complex challenges.
*   **Understanding natural language:** Interpreting and responding to human speech or text.
*   **Recognizing patterns:** Identifying trends and relationships in data.
*   **Making decisions:** Choosing the best course of action based on available information.

You encounter AI every day in things like recommendation systems (Netflix, Amazon), virtual assistants (Siri, Alexa), spam filters, self-driving cars, and even the way your phone su